In [ ]:
!pip install konlpy
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661624 sha256=14d7ebe3dc2ed0edc38d9ecad88b0f6848adb19bc0f1d6c28444a90e893fb200
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 29.3 MB/s eta 0:00:00
     ━━━━

In [ ]:
from konlpy.tag import Komoran
komoran = Komoran()

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model = AutoModelForMaskedLM.from_pretrained("monologg/kobert")

Some weights of BertForMaskedLM were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from konlpy.tag import Komoran
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
import itertools
import numpy as np
import time, datetime
import matplotlib.pyplot as plt

In [ ]:
# GPU 사용
device = torch.device("cuda:0")

In [ ]:
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [ ]:
# 형태소 분석 후 언어정보점수 추가
def add_linguistic_score(sentence, a, b, c, d, e, f, g):
    pos = komoran.pos(sentence)
    score = 0
    for p in pos:
        if p[1] == 'NNP': # 고유 명사
            score += a
        elif p[1] == 'NNG': # 일반 명사
            score += b
        elif p[1] == 'VV': # 동사 -> 서술어
            score += c
        elif p[1] == 'SL': # 외국어
            score += d
        elif p[1] == 'JKS': # 주격 조사 -> 주어
            score += e
        elif p[1] == 'JKO': # 목적격 조사 -> 목적어
            score += f
    quoted_words = re.findall(r"'(.*?)'", sentence)
    for quoted_word in quoted_words: # 따옴표 안의 단어
        score += g
    return score

# 문장 어절 단위로 토큰화
def word_tokenizer(sentence):
    return sentence.split(' ')

# 따옴표와 따옴표를 포함하는 문장을 하나의 토큰으로
def process_quoted_words(tokens):
    processed_tokens = []
    quoted_word = ""
    in_quote = False

    for token in tokens:
        if "'" in token:
            if in_quote:
                quoted_word += " " + token
                processed_tokens.append(quoted_word)
                quoted_word = ""
                in_quote = False
            else:
                quoted_word = token
                in_quote = True
        else:
            if in_quote:
                quoted_word += " " + token
            else:
                processed_tokens.append(token)
    return processed_tokens

# 압축 문장 생성
def compress_sentence(sentence, a, b, c, d, e, f, g):

    token = word_tokenizer(sentence)
    token = process_quoted_words(token)

    # 압축 문장 후보군 생성 및 언어정보점수 계산
    max_n = len(token) - 4
    # max_n = 4
    compressed_candidates = []
    for n in range(1, max_n + 1):
        for i in range(len(token) - n + 1):
            compressed_tokens = token[:i] + token[i + n:]
            compressed_sentence = " ".join(compressed_tokens)
            # 형태소 점수 계산
            score = add_linguistic_score(compressed_sentence, a, b, c, d, e, f, g)
            # 각 요약 문장 후보에 형태소 점수 추가
            compressed_candidates.append((compressed_sentence, score, n))

    # perplexity 계산
    def calculate_perplexity_score(sentence):

        # KoBERT 모델이 이해할 수 있는 형태로 tokenize
        tokens = tokenizer.tokenize(sentence)
        token_ids = tokenizer.convert_tokens_to_ids(tokens) # tokenized word -> 정수 인덱스

        # [MASK] 토큰 위치 찾기
        mask_token_index = token_ids.index(tokenizer.mask_token_id)

        # 입력값과 출력값 생성
        input_ids = torch.tensor([token_ids]).to(device)
        outputs = model(input_ids) # 예측값
        predictions = outputs[0]

        # [MASK] 토큰 위치에 대한 예측값 추출
        masked_predictions = predictions[0, mask_token_index]

        # softmax 함수를 이용하여 확률값을 확률 분포로 변환
        probs = torch.softmax(masked_predictions, dim=-1)

        # perplexity 계산
        perplexity = torch.exp(torch.mean(torch.log(probs)))

        return perplexity.item()

    perplexity_scores = []
    compressed_candidates_with_score = []

    for n in range(1, max_n + 1):
        for i in range(len(token) - n + 1):
            mask_idx = list(range(i, i + n))
            masked_tokens = list(token)
            for j in mask_idx:
                masked_tokens[j] = "[MASK]"
            masked_sentence = " ".join(masked_tokens)

            perplexity_score = calculate_perplexity_score(masked_sentence) # perplexity 계산
            linguistic_score = compressed_candidates[i][1] # 형태소 점수 가져오기
            final_score = 1/perplexity_score * linguistic_score #* (-1) # 최종 점수 계산

            # 최종 점수 저장
            perplexity_scores.append(perplexity_score)
            compressed_candidates_with_score.append((re.sub(r'\[MASK\]\s*', '', masked_sentence), final_score, n))

    # 최종 요약 문장 선택 (최종 점수가 가장 낮은)
    compressed_candidates_with_score_sorted = sorted(compressed_candidates_with_score, key=lambda x: x[1], reverse = True)

    if compressed_candidates_with_score_sorted:
      final_compressed_sentence = re.sub(r'\[MASK\]\s*', '', compressed_candidates_with_score_sorted[0][0])
      selected_n = compressed_candidates_with_score_sorted[0][2]
    else:
      # 리스트가 비어있을 경우에 대한 예외 처리 또는 기본값 설정
      final_compressed_sentence = "No compressed sentence available"
      selected_n = 0

    return final_compressed_sentence # 최종 요약 문장 반환

# 코사인 유사도 계산을 위한 모델 임베딩 함수 정의
def encode_sentence(sentence):

    # 문장을 토큰화하고 토큰을 ID로 변환
    tokens = tokenizer.tokenize(sentence)

    if len(tokens) == 0 :
        return None

    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    # 모델 입력으로 사용할 수 있도록 텐서로 변환하고 GPU로 이동
    input_ids = torch.tensor([token_ids]).to(device)

    # 모델에 입력을 전달하고 출력을 얻음
    with torch.no_grad():
        outputs = model(input_ids)

    # 모델의 마지막 hidden state에서 평균을 계산하여 문장 임베딩 얻음
    hidden_states = outputs[0]
    sentence_embedding = torch.mean(hidden_states, dim=1).squeeze().cpu().numpy()

    sentence_embedding = sentence_embedding.reshape(1,-1)

    return sentence_embedding

In [ ]:
# main

if __name__ == "__main__":
    start_time = time.time()
    print(datetime.datetime.now())

    # a, b, c, d, e, f, g 값 선택
    a = 0.0001
    b = 0.0001
    c = 0.0001
    d = 0.0001
    e = 0.0001
    f = 0.0001
    g = 0.0001

    # 'source_file.txt'와 'target_file.txt'에서 문장 쌍 가져오기
    # 파일 경로 설정
    source_file = '/content/drive/MyDrive/23-2/졸업논문/DATA/DATA/source_축약 전 원문 파일.txt'
    target_file = '/content/drive/MyDrive/23-2/졸업논문/DATA/DATA/target_정답 축약 문장 파일.txt'

    source_sentences = []
    target_sentences = []

    with open(source_file,'r',encoding='utf-8') as r:
      source_sentences = r.readlines()

    with open(target_file, 'r', encoding='utf-8') as r :
      target_sentences = r.readlines()

    # 각 경우의 수에 대해 문장 압축을 수행하고 결과 저장
    all_cosine_similarities = []

    for i in range(len(source_sentences)): # 각 문장 쌍에 대해서

        input_sentence = source_sentences[i].strip()
        target_sentence = target_sentences[i].strip()

        print(int(time.time()-start_time), "초 소요", datetime.datetime.now())

        compressed_sentence = compress_sentence(input_sentence, a, b, c, d, e, f, g) # 문장 요약
        target_embedding = encode_sentence(target_sentence ) # 정답 요약 문장
        compressed_embedding = encode_sentence(compressed_sentence) # 모델 요약 문장
        similarity = cosine_similarity(target_embedding, compressed_embedding)[0][0] # 유사도 계산

        # 현재 문장 쌍에 대해 코사인 유사도, parameter를 리스트에 추가
        all_cosine_similarities.append(similarity)

# 파일로 저장
with open(f'/content/drive/MyDrive/23-2/졸업논문/parameter_p역수_all_cosine_similarities.txt', 'w', encoding="utf-8") as result_file:
    for item in all_cosine_similarities:
        similarity = item
        result_file.write(f"{similarity}\n")

2023-10-17 08:31:03.515579
0 초 소요 2023-10-17 08:31:03.528061
0 초 소요 2023-10-17 08:31:03.815278
1 초 소요 2023-10-17 08:31:05.181180
1 초 소요 2023-10-17 08:31:05.362133
2 초 소요 2023-10-17 08:31:06.490051
5 초 소요 2023-10-17 08:31:08.881315
5 초 소요 2023-10-17 08:31:09.472080
7 초 소요 2023-10-17 08:31:11.367559
8 초 소요 2023-10-17 08:31:12.013269
8 초 소요 2023-10-17 08:31:12.470785
9 초 소요 2023-10-17 08:31:13.466401
10 초 소요 2023-10-17 08:31:13.557587
10 초 소요 2023-10-17 08:31:14.244102
10 초 소요 2023-10-17 08:31:14.504001
13 초 소요 2023-10-17 08:31:17.414382
15 초 소요 2023-10-17 08:31:18.868886
15 초 소요 2023-10-17 08:31:19.382932
17 초 소요 2023-10-17 08:31:20.816329
17 초 소요 2023-10-17 08:31:20.904738
17 초 소요 2023-10-17 08:31:21.177267
19 초 소요 2023-10-17 08:31:22.610006
19 초 소요 2023-10-17 08:31:22.793896
19 초 소요 2023-10-17 08:31:23.471037
22 초 소요 2023-10-17 08:31:25.839303
22 초 소요 2023-10-17 08:31:26.237161
27 초 소요 2023-10-17 08:31:30.793408
29 초 소요 2023-10-17 08:31:32.722669
29 초 소요 2023-10-17 08:31:32.830830
29 초

In [ ]:
len(all_cosine_similarities)

In [ ]:
len(source_sentences)